# A better linear model

In [58]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_selection import mutual_info_regression, SequentialFeatureSelector, RFECV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, PoissonRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer
from sklearn.model_selection import KFold, RepeatedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder, StandardScaler, RobustScaler
from sklearn.tree import DecisionTreeRegressor

In [59]:
X = pd.read_csv('../data_students/labeled_data/X_train.csv')
X_test  = pd.read_csv('../data_students/labeled_data/X_test.csv')
y = pd.read_csv('../data_students/labeled_data/y_train.csv', header=None)
y_test  = pd.read_csv('../data_students/labeled_data/y_test.csv', header=None)

# Getting rid of the column 'img_filename'
X = X.drop(columns=['img_filename'])
X_test  = X_test.drop(columns=['img_filename'])

y      = y.squeeze()
y_test = y_test.squeeze()

profession_categories = [['food production', 'administration and governance', 'services', 'ressource extraction', 'craftsmanship', 'manufacturing']]
ordinal_categories = ['sarsaparilla', 'smurfberry liquor', 'smurfin donuts']
normally_distributed_categories = ['age', 'blood pressure', 'calcium', 'cholesterol', 'hemoglobin', 'height', 'potassium', 'vitamin D', 'weight']

# Pre-processing

### Transformation

**Outliers handling**

Cholesterol has really bad outliers, so we remove them and it improves the model.

In [60]:
cholesterol_indexes = X[X['cholesterol'] < 5].index
X = X.drop(cholesterol_indexes).reset_index(drop=True)
y = y.drop(cholesterol_indexes).reset_index(drop=True)

**Ordinal encoding**

Since sarsaparilla, smurfberry liquor and smurfin donuts have some ordinality (from very low to very high) we can transform it with values from 0 to 5.

In [61]:
ordinal_encoder = OrdinalEncoder(categories=[['Very low', 'Low', 'Moderate', 'High', 'Very high']])
for category in ordinal_categories:
    X[category]      = ordinal_encoder.fit_transform(X[[category]])
    X_test[category] = ordinal_encoder.transform(X_test[[category]])

**One-hot encoder** for Profession category

We create a vector of values which are set to zeros except for the profession value of the item.

In [62]:
one_hot_encoder = OneHotEncoder(categories=profession_categories, sparse_output=False)

profession_encoded_train = one_hot_encoder.fit_transform(X[['profession']])
profession_encoded_train_df = pd.DataFrame(profession_encoded_train, columns=one_hot_encoder.get_feature_names_out(['profession']))
X = pd.concat([X.drop('profession', axis=1), profession_encoded_train_df], axis=1)

profession_encoded_test = one_hot_encoder.transform(X_test[['profession']])
profession_encoded_test_df = pd.DataFrame(profession_encoded_test, columns=one_hot_encoder.get_feature_names_out(['profession']))
X_test = pd.concat([X_test.drop('profession', axis=1), profession_encoded_test_df], axis=1)

### Standardization
We should check for the MinMaxScaler because there is no improvement compare to StandardScaler for the ordinal_categories.

Output is not scaled since it is already $\in [0, 1]$, being a probability.

In [63]:
ordinal_scaler = MinMaxScaler()
X[ordinal_categories] = pd.DataFrame(ordinal_scaler.fit_transform(X[ordinal_categories]))
X_test[ordinal_categories] = pd.DataFrame(ordinal_scaler.transform(X_test[ordinal_categories]))

normal_scaler = RobustScaler()
X[normally_distributed_categories] = pd.DataFrame(normal_scaler.fit_transform(X[normally_distributed_categories]))
X_test[normally_distributed_categories] = pd.DataFrame(normal_scaler.transform(X_test[normally_distributed_categories]))

# Feature selection

### Benchmark

In [64]:
def selection_performance(y_pred, features):
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Selected features: ")
    for feature in features:
        print(f"- {feature}")
    print("----------------------------------")
    print(f"RMSE: \t\t\t{rmse:.6f}")
    print(f"Mean Absolute Error: \t{mae:.6f}")
    print(f"R-squared score: \t{r2:.6f}")

In [65]:
model = LinearRegression().fit(X[X.columns], y)
y_pred = model.predict(X_test[X.columns])
selection_performance(y_pred, X.columns)

Selected features: 
- age
- blood pressure
- calcium
- cholesterol
- hemoglobin
- height
- potassium
- sarsaparilla
- smurfberry liquor
- smurfin donuts
- vitamin D
- weight
- profession_food production
- profession_administration and governance
- profession_services
- profession_ressource extraction
- profession_craftsmanship
- profession_manufacturing
----------------------------------
RMSE: 			0.077627
Mean Absolute Error: 	0.056825
R-squared score: 	0.311337


### Tests for selection

**Correlation filter** determines that using the eleventh most correlated features to the output gives us the best result.

In [66]:
n_features = 11
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)

correlation_filter_scores = np.zeros(X.shape[1])
for train_idx, val_idx in rkf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    corr = X_train.corrwith(y_train).abs()
    correlation_filter_scores += corr
correlation_filter_scores /= (rkf.get_n_splits())
top_features_indices = np.argsort(correlation_filter_scores)[-n_features:]
best_features_corr = X.columns[top_features_indices]

y_pred = LinearRegression().fit(X[best_features_corr], y).predict(X_test[best_features_corr])
selection_performance(y_pred, best_features_corr)


Selected features: 
- profession_services
- profession_administration and governance
- smurfberry liquor
- potassium
- height
- age
- smurfin donuts
- cholesterol
- sarsaparilla
- weight
- blood pressure
----------------------------------
RMSE: 			0.077385
Mean Absolute Error: 	0.056731
R-squared score: 	0.315622


**Mutual information filtering** gives us little information. 16 features are required, only removing ...

In [67]:
n_features = 16

rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)

mutual_info_scores = np.zeros(X.shape[1])
for train_idx, val_idx in rkf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    mi = mutual_info_regression(X_train, y_train, random_state=42)
    mutual_info_scores += mi
mutual_info_scores /= (rkf.get_n_splits())
top_features_indices = np.argsort(mutual_info_scores)[-n_features:]
best_features = X.columns[top_features_indices]

y_pred = LinearRegression().fit(X[best_features], y).predict(X_test[best_features])
selection_performance(y_pred, best_features)

Selected features: 
- profession_administration and governance
- height
- profession_food production
- profession_manufacturing
- smurfin donuts
- profession_ressource extraction
- smurfberry liquor
- profession_services
- hemoglobin
- cholesterol
- age
- sarsaparilla
- potassium
- calcium
- weight
- blood pressure
----------------------------------
RMSE: 			0.077626
Mean Absolute Error: 	0.056826
R-squared score: 	0.311351


**Maximum relevance and minimum redundancy** is basically just removing smurfin donuts and keeping weight or conversely, as those are the only features really correlated.

In [68]:
n_features = 16
corr_threshold = 0.6

rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)
mutual_info_scores = np.zeros(X.shape[1])
for train_idx, val_idx in rkf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    mi = mutual_info_regression(X_train, y_train, random_state=42)
    mutual_info_scores += mi
mutual_info_scores /= (rkf.get_n_splits())

mi_series = pd.Series(mutual_info_scores, index=X.columns)
ranked_features = list(mi_series.sort_values(ascending=False).index)

corr = X.corr()

selected_features = []
for feature in ranked_features:
    if len(selected_features) == n_features:
        break
    elif len(selected_features) == 0:
        selected_features.append(feature)
    else:
        feature_is_redundant = False
        for selected_feature in selected_features:
            if np.abs(corr[feature][selected_feature]) > corr_threshold:
                feature_is_redundant = True
                break
        if not feature_is_redundant:
            selected_features.append(feature)

best_features = selected_features

y_pred = LinearRegression().fit(X[best_features], y).predict(X_test[best_features])
selection_performance(y_pred, best_features)

Selected features: 
- blood pressure
- weight
- calcium
- potassium
- sarsaparilla
- age
- cholesterol
- hemoglobin
- profession_services
- smurfberry liquor
- profession_ressource extraction
- profession_manufacturing
- profession_food production
- height
- profession_administration and governance
- vitamin D
----------------------------------
RMSE: 			0.077655
Mean Absolute Error: 	0.056826
R-squared score: 	0.310824


**Forward wrapper** gives no concluant result.

In [69]:
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)
sfs = SequentialFeatureSelector(LinearRegression(), n_features_to_select='auto', direction='forward', cv=rkf)
sfs.fit(X, y)
best_features = X.columns[sfs.support_]

y_pred = LinearRegression().fit(X[best_features],y).predict(X_test[best_features])
selection_performance(y_pred, best_features)

Selected features: 
- age
- blood pressure
- cholesterol
- height
- potassium
- sarsaparilla
- smurfberry liquor
- weight
- profession_ressource extraction
----------------------------------
RMSE: 			0.077596
Mean Absolute Error: 	0.057068
R-squared score: 	0.311880


**Backward wrapper**

In [70]:
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)
sfs = SequentialFeatureSelector(LinearRegression(), n_features_to_select='auto', direction='backward', cv=rkf)
sfs.fit(X, y)
best_features = X.columns[sfs.support_]

y_pred = LinearRegression().fit(X[best_features],y).predict(X_test[best_features])
selection_performance(y_pred, best_features)

Selected features: 
- age
- blood pressure
- cholesterol
- height
- potassium
- sarsaparilla
- smurfberry liquor
- weight
- profession_ressource extraction
----------------------------------
RMSE: 			0.077596
Mean Absolute Error: 	0.057068
R-squared score: 	0.311880


**Decision Tree** For most relevant features

In [71]:
dt = DecisionTreeRegressor(max_depth=7) # best depth, tried different ones.
dt.fit(X, y)

y_pred = dt.predict(X_test)
rmse = np.sqrt(np.mean((y_pred-y_test.values.ravel())**2))

feature_importances = pd.Series(dt.feature_importances_, index=X.columns)

print("Feature importances:")
for feature,feature_importance in feature_importances.items():
    print(f"- {feature:20} {feature_importance:5.3f}")
print("--------------------------------------")
print(f"Decision tree RMSE: {rmse:5.3f}")

Feature importances:
- age                  0.098
- blood pressure       0.490
- calcium              0.021
- cholesterol          0.059
- hemoglobin           0.034
- height               0.024
- potassium            0.122
- sarsaparilla         0.025
- smurfberry liquor    0.057
- smurfin donuts       0.011
- vitamin D            0.002
- weight               0.042
- profession_food production 0.000
- profession_administration and governance 0.000
- profession_services  0.016
- profession_ressource extraction 0.000
- profession_craftsmanship 0.000
- profession_manufacturing 0.000
--------------------------------------
Decision tree RMSE: 0.083


In [72]:
n_features = 18
n_best = 2
rmse_best = np.inf
for i in range (2, n_features+1):
    best_features = list(feature_importances.sort_values(ascending=False)[:i].index)
    y_pred = LinearRegression().fit(X[best_features],y).predict(X_test[best_features])
    #print(f"Number of features: {i}")
    selection_performance(y_pred, best_features)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    if rmse < rmse_best:
        rmse_best = rmse
        n_best = i
    print()

print(f"Best number of features: {n_best}")
    

# best_features = list(feature_importances.sort_values(ascending=False)[:n_features].index)
# y_pred = LinearRegression().fit(X[best_features],y).predict(X_test[best_features])
# selection_performance(y_pred, best_features)

Selected features: 
- blood pressure
- potassium
----------------------------------
RMSE: 			0.082283
Mean Absolute Error: 	0.059738
R-squared score: 	0.226233

Selected features: 
- blood pressure
- potassium
- age
----------------------------------
RMSE: 			0.082290
Mean Absolute Error: 	0.059751
R-squared score: 	0.226106

Selected features: 
- blood pressure
- potassium
- age
- cholesterol
----------------------------------
RMSE: 			0.082012
Mean Absolute Error: 	0.059687
R-squared score: 	0.231326

Selected features: 
- blood pressure
- potassium
- age
- cholesterol
- smurfberry liquor
----------------------------------
RMSE: 			0.081036
Mean Absolute Error: 	0.059167
R-squared score: 	0.249516

Selected features: 
- blood pressure
- potassium
- age
- cholesterol
- smurfberry liquor
- weight
----------------------------------
RMSE: 			0.078908
Mean Absolute Error: 	0.057616
R-squared score: 	0.288418

Selected features: 
- blood pressure
- potassium
- age
- cholesterol
- smurfberr

**Recursive feature elimination**

In [73]:
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)
rfecv = RFECV(estimator=LinearRegression(), step=1, cv=rkf, scoring='neg_mean_squared_error')
rfecv.fit(X, y)
best_features = X.columns[rfecv.support_]

y_pred = rfecv.predict(X_test)
selection_performance(y_pred, best_features)

Selected features: 
- age
- blood pressure
- cholesterol
- height
- potassium
- sarsaparilla
- smurfberry liquor
- smurfin donuts
- weight
- profession_ressource extraction
- profession_manufacturing
----------------------------------
RMSE: 			0.077594
Mean Absolute Error: 	0.056910
R-squared score: 	0.311906


In [74]:
# Update training data and test data 
X = X[best_features_corr]
X_test = X_test[best_features_corr]

# Model Selection

#### Validation set

In [75]:
X_learn, X_val, y_learn, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [76]:
def find_best_model_kfold(X, y, models, k=5):
    best_model = None
    best_rmse = float('inf')
    
    # KFold avec k folds (par défaut, 5)
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
    for model in models:
        fold_rmse = []
        
        # KFold splitting
        for train_index, val_index in kf.split(X):
            # Utilisation de .iloc pour indexer correctement le DataFrame
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]
            
            model.fit(X_train, y_train)
            y_pred = model.predict(X_val)
            rmse = np.sqrt(mean_squared_error(y_val, y_pred))
            fold_rmse.append(rmse)
        
        # Moyenne des RMSE sur les k folds
        mean_rmse = np.mean(fold_rmse)
        print(f"Model: {model}, Mean RMSE across {k} folds: {mean_rmse}")
        
        if mean_rmse < best_rmse:
            best_rmse = mean_rmse
            best_model = model
    
    return best_model

# Modèles linéaires à tester
models = [LinearRegression(), Lasso(), Ridge(), ElasticNet(), PoissonRegressor()]

# Trouver le meilleur modèle avec KFold
best_model = find_best_model_kfold(X_learn, y_learn, models)

print("\nBest model:", best_model)

# Évaluation finale sur l'ensemble de test
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"RMSE on the test set: {rmse:.6f}")
print(f"Mean Absolute Error on the test set: {mae:.6f}")
print(f"R-squared score on the test set: {r2:.6f}")

Model: LinearRegression(), Mean RMSE across 5 folds: 0.07806606174131595
Model: Lasso(), Mean RMSE across 5 folds: 0.08896551947291972
Model: Ridge(), Mean RMSE across 5 folds: 0.07804962640804718
Model: ElasticNet(), Mean RMSE across 5 folds: 0.08896551947291972
Model: PoissonRegressor(), Mean RMSE across 5 folds: 0.08772361102298927

Best model: Ridge()
RMSE on the test set: 0.077057
Mean Absolute Error on the test set: 0.057241
R-squared score on the test set: 0.321396
